# Padrão Strategy
Ao utilizar este padrão podemos passar como parâmetro uma função, ou melhor, uma estratégia, para outro método.

Isso permite remover vários comandos condicionais, uma vez que não precisamos verificar manualmente qual foi a estratégia passada como parâmetro.

Nota: diferença entre função e método: um método pertence a uma classe e uma função não pertence necessariamente a uma classe.

**Duck typing:** é um estilo de tipagem em que os métodos e propriedades de um objeto determinam a semântica válida, em vez de sua herança de uma classe particular ou implementação de uma interface explicita. Não importa a instância que passamos, contanto que ela tenha o método calcula.

## Quando usar?
O padrão Strategy é muito útil quando temos um conjunto de algoritmos similares, e precisamos alternar entre eles em diferentes pedaços da aplicação. O Strategy nos oferece uma maneira flexível para escrever diversos algoritmos diferentes, e de passar esses algoritmos para classes clientes que precisam deles. Esses clientes desconhecem qual é o algoritmo "real" que está sendo executado, e apenas manda o algoritmo rodar. Isso faz com que o código da classe cliente fique bastante desacoplado das implementações dos algoritmos, possibilitando assim com que esse cliente consiga trabalhar com N diferentes algoritmos sem precisar alterar o seu código.


### Utilize o padrão Strategy quando você quer usar diferentes variantes de um algoritmo dentro de um objeto e ser capaz de trocar de um algoritmo para outro durante a execução.

 O padrão Strategy permite que você altere indiretamente o comportamento de um objeto durante a execução ao associá-lo com diferentes sub-objetos que pode fazer sub-tarefas específicas em diferentes formas.

### Utilize o Strategy quando você tem muitas classes parecidas que somente diferem na forma que elas executam algum comportamento.

 O padrão Strategy permite que você extraia o comportamento variante para uma hierarquia de classe separada e combine as classes originais em uma, portando reduzindo código duplicado.

### Utilize o padrão para isolar a lógica do negócio de uma classe dos detalhes de implementação de algoritmos que podem não ser tão importantes no contexto da lógica.

 O padrão Strategy permite que você isole o código, dados internos, e dependências de vários algoritmos do restante do código. Vários clientes podem obter uma simples interface para executar os algoritmos e trocá-los durante a execução do programa.

### Utilize o padrão quando sua classe tem um operador condicional muito grande que troca entre diferentes variantes do mesmo algoritmo.

 O padrão Strategy permite que você se livre dessa condicional ao extrair todos os algoritmos para classes separadas, todos eles implementando a mesma interface. O objeto original delega a execução de um desses objetos, ao invés de implementar todas as variantes do algoritmo.

In [1]:
# Context

class Context:
    def calcular(self, receita, strategy):
        return strategy.calcular(receita)
    


In [2]:
from abc import ABC, abstractmethod

class Interface:
    @abstractmethod
    def calcular(self, receita):
        """Realiza algum cálculo"""
        

class Desconto(Interface):
    def calcular(self, receita):
        return receita.valor * 0.95
    
    
class Juros(Interface):
    def calcular(self, receita):
        return receita.valor * 1.05

In [3]:
class Receita:
    def __init__(self, valor):
        self.__valor = valor

    @property
    def valor(self):
        return self.__valor

In [4]:
receita = Receita(1000)
context = Context()

In [5]:
# Aplicar desconto

desconto = context.calcular(receita, Desconto())
print(desconto)

950.0


In [6]:
# Aplicar juros

juros = context.calcular(receita, Juros())
print(juros)

1050.0


# Simulação de uso real

Esse Pattern é utilizado para códigos similares, que devem ser trocados dinamicamente.

Pode reduzir os `if's` de uma condição. 

In [7]:
# exemplo

# checagem se o produto é válido utilizando if's
def produto_checker(produto):
    is_valid = True
    if len(produto) < 3:
        is_valid = False
    elif "valor" not in produto or "nome" not in produto or "categoria" not in produto:
        is_valid = False
    elif produto.get("valor") > 5000:
        is_valid = False
    elif produto.get("categoria") not in ("eletrodomestico", "eletroportateis", "jardinagem"):
        is_valid = False
        
    return is_valid

In [8]:
geladeira = {
    "valor": 2000,
    "nome": "geladeira",
    "categoria": "eletrodomestico",
}

produto_checker(geladeira)

True

In [9]:
carro = {
    "valor": 50000,
    "nome": "carro",
    "categoria": "automovel",
}

produto_checker(carro)

False

# Critica

Podemos ver que o código ficou repetitivo e dificil refatoração ou inclusão de novas regras de validação.

Além disso, outras melhorias de código como aplicar assíncronicidade ou outras tarefas, iriam complicar extremamente o código.


Agora vamos utilizar o design pattern `Strategy` para refatorar o trecho de código.

In [10]:
class Produto:
    
    PRODUTO_ITEMS = ["valor", "nome", "categoria"]
    
    def __init__(self, nome: str, valor: int, categoria: str):
        self.__valor = valor
        self.__nome = nome
        self.__categoria = categoria
        self.__pago = None
        
    @property
    def valor(self):
        return self.__valor
    
    @property
    def nome(self):
        return self.__nome
    
    @property
    def categoria(self):
        return self.__categoria.upper()
    
    @property
    def pago(self):
        return self.__pago
    
    @pago.setter
    def pago(self, new_status: bool):
        self.__pago = new_status
    
    def __getattr__(self, obj):
        if obj not in self.PRODUTO_ITEMS:
            return None
        
    def __repr__(self):
        return (
            f"Nome: {self.nome}\n"
            f"Valor: {self.valor}\n"
            f"Categoria: {self.categoria}\n"
            f"Pago: {self.pago}"
        )

In [11]:
from abc import ABC, abstractmethod


class Checker:
    def check_it(self, produto, strategy):
        return strategy.check(produto)
    
    
class Interface:
    def check(self, produto):
        pass
    

class ProdutoNameLenCheck(Interface):
    def check(self, produto):
        return len(produto.nome) < 10
    

class ValorCheck(Interface):
    def check(self, produto):
        valor = produto.valor
        return False if not valor else valor < 5000


class CategoriaCheck(Interface):
    ALLOWED_CATEGORIES = ("ELETRODOMESTICO", "ELETROPORTATIL", "JARDINAGEM")
    
    def check(self, produto):
        return produto.categoria.upper() in self.ALLOWED_CATEGORIES


In [12]:
produto = Produto("geladeira", 2000, "eletrodomestico")
produto

Nome: geladeira
Valor: 2000
Categoria: ELETRODOMESTICO
Pago: None

# Eliminando os if's

Agora o código irá chamar todas classes que herdam de interface, e chamar o método check passando o produto

In [13]:
ProdutoNameLenCheck().check(produto)

True

In [14]:
ValorCheck().check(produto)

True

In [15]:
CategoriaCheck().check(produto)

True

# Simplificando usando o all

In [16]:
def check_all(product):
    return all([checker().check(product) for checker in (ProdutoNameLenCheck, ValorCheck, CategoriaCheck)])

check_all(produto)

True

In [17]:
# Retornará falso porque ferramenta não é uma categoria válida

produto2 = Produto("furadeira", 500, "ferramenta")
produto2

Nome: furadeira
Valor: 500
Categoria: FERRAMENTA
Pago: None

In [18]:
check_all(produto2)

False

# Refactorando o código para torná-lo ainda mais genérico e facilitar a adição de regras

In [19]:
# Filtra todas as classes sem a necessidade de passar manualmente
# Cada nova classe será filtrada automaticamente

classes_str = [cls for cls in dir() if cls.endswith("Check")]
classes_str

['CategoriaCheck', 'ProdutoNameLenCheck', 'ValorCheck']

In [20]:
# Selecionando as classes do módulo

module = __import__(__name__)
classes = [getattr(module, class_name) for class_name in classes_str]
classes

[__main__.CategoriaCheck, __main__.ProdutoNameLenCheck, __main__.ValorCheck]

In [21]:
# Agora cada novo validador será adicionado automaticamente a cada execução do modulo

def generic_check_all(product):
    return all([checker().check(product) for checker in classes])

In [22]:
generic_check_all(produto)

True

In [23]:
generic_check_all(produto2)

False

# Adicionando nova classe para provar o conceito

In [24]:
class PagamentoCheck(Interface):    
    def check(self, produto):
        return produto.pago is True

In [25]:
PagamentoCheck().check(produto)

False

In [26]:
# Precisamos reexecutar no jupiter pois estamos ainda na mesma execução do programa
# Em programas dinamicos, não seria necessário

classes_str = [cls for cls in dir() if cls.endswith("Check")]
module = __import__(__name__)
classes = [getattr(module, class_name) for class_name in classes_str]

classes

[__main__.CategoriaCheck,
 __main__.PagamentoCheck,
 __main__.ProdutoNameLenCheck,
 __main__.ValorCheck]

In [27]:
def generic_check_all(product):
    return all([checker().check(product) for checker in classes])

# Agora produto irá falhar, pois ainda não foi pago
generic_check_all(produto)

False

In [28]:
produto

Nome: geladeira
Valor: 2000
Categoria: ELETRODOMESTICO
Pago: None

In [29]:
# Agora vamos alterar pago para True
produto.pago = True
produto

Nome: geladeira
Valor: 2000
Categoria: ELETRODOMESTICO
Pago: True

In [30]:
generic_check_all(produto)

True